# Joining Data with pandas - 4

## `merge_ordered()`

In [1]:
import pandas as pd
import numpy as np

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
sp500 = pd.read_csv("data/sp500.csv")
gdp = pd.read_csv("data/WorldBank_GDP.csv")

<IPython.core.display.Javascript object>

In [3]:
sp500["date"] = sp500["Date"].str.split("-").apply(lambda x: x[0])
sp500["date"] = sp500["date"].astype(int)

<IPython.core.display.Javascript object>

In [4]:
# Use merge_ordered() to merge gdp and sp500, interpolate missing value
gdp_sp500 = pd.merge_ordered(
    gdp, sp500, left_on="Year", right_on="date", how="left", fill_method="ffill"
)

# Subset the gdp and Adj Close columns
gdp_close = gdp_sp500[["GDP", "Adj Close"]]

# Print gdp_returns correlation
print(gdp_close.corr())

                GDP  Adj Close
GDP        1.000000  -0.000537
Adj Close -0.000537   1.000000


<IPython.core.display.Javascript object>

```python
pandas.merge_ordered(left, right, on=None, left_on=None, right_on=None, left_by=None, right_by=None, fill_method=None, suffixes='_x', '_y', how='outer')
```

In [5]:
df1 = pd.DataFrame(
    {
        "key": ["a", "c", "e", "a", "c", "e"],
        "lvalue": [1, 2, 3, 1, 2, 3],
        "group": ["a", "a", "a", "b", "b", "b"],
    }
)
df1

,key,lvalue,group
0,a,1,a
1,c,2,a
2,e,3,a
3,a,1,b
4,c,2,b
5,e,3,b


<IPython.core.display.Javascript object>

In [6]:
df2 = pd.DataFrame({"key": ["b", "c", "d"], "rvalue": [1, 2, 3]})
df2

,key,rvalue
0,b,1
1,c,2
2,d,3


<IPython.core.display.Javascript object>

In [7]:
pd.merge_ordered(df1, df2)

,key,lvalue,group,rvalue
0,a,1.0,a,NaN
1,a,1.0,b,NaN
2,b,NaN,NaN,1.0
3,c,2.0,a,2.0
4,c,2.0,b,2.0
5,d,NaN,NaN,3.0
6,e,3.0,a,NaN
7,e,3.0,b,NaN


<IPython.core.display.Javascript object>

In [8]:
pd.merge_ordered(df1, df2, left_by="group")

,key,lvalue,group,rvalue
0,a,1.0,a,NaN
1,b,NaN,a,1.0
2,c,2.0,a,2.0
3,d,NaN,a,3.0
4,e,3.0,a,NaN
5,a,1.0,b,NaN
6,b,NaN,b,1.0
7,c,2.0,b,2.0
8,d,NaN,b,3.0
9,e,3.0,b,NaN


<IPython.core.display.Javascript object>

In [9]:
pd.merge_ordered(df1, df2, fill_method="ffill", left_by="group")

,key,lvalue,group,rvalue
0,a,1,a,NaN
1,b,1,a,1.0
2,c,2,a,2.0
3,d,2,a,3.0
4,e,3,a,3.0
5,a,1,b,NaN
6,b,1,b,1.0
7,c,2,b,2.0
8,d,2,b,3.0
9,e,3,b,3.0


<IPython.core.display.Javascript object>

In [10]:
pd.merge(df1, df2, on="key", how="outer")

,key,lvalue,group,rvalue
0,a,1.0,a,NaN
1,a,1.0,b,NaN
2,c,2.0,a,2.0
3,c,2.0,b,2.0
4,e,3.0,a,NaN
5,e,3.0,b,NaN
6,b,NaN,NaN,1.0
7,d,NaN,NaN,3.0


<IPython.core.display.Javascript object>

When to use `merge_ordered()`:

* Ordered data/time series
* Filling in missing values

## `merge_asof()`


```python
pandas.merge_asof(left, right, on=None, left_on=None, right_on=None, left_index=False, right_index=False, by=None, left_by=None, right_by=None, suffixes='_x', '_y', tolerance=None, allow_exact_matches=True, direction='backward')
```

Perform an asof merge.

This is similar to a left-join except that we match on nearest key rather than equal keys. Both DataFrames must be sorted by the key.

For each row in the left DataFrame:
* A “backward” search selects the last row in the right DataFrame whose ‘on’ key is less than or equal to the left’s key.
* A “forward” search selects the first row in the right DataFrame whose ‘on’ key is greater than or equal to the left’s key.
* A “nearest” search selects the row in the right DataFrame whose ‘on’ key is closest in absolute distance to the left’s key.

Optionally match on equivalent keys with ‘by’ before searching with ‘on’.

In [11]:
left = pd.DataFrame({"a": [1, 5, 10], "left_val": ["a", "b", "c"]})
left

,a,left_val
0,1,a
1,5,b
2,10,c


<IPython.core.display.Javascript object>

In [12]:
right = pd.DataFrame({"a": [1, 2, 3, 6, 7], "right_val": [1, 2, 3, 6, 7]})
right

,a,right_val
0,1,1
1,2,2
2,3,3
3,6,6
4,7,7


<IPython.core.display.Javascript object>

In [13]:
pd.merge_asof(left, right, on="a")

,a,left_val,right_val
0,1,a,1
1,5,b,3
2,10,c,7


<IPython.core.display.Javascript object>

`allow_exact_matches`
* If True, allow matching with the same ‘on’ value (i.e. less-than-or-equal-to / greater-than-or-equal-to)
* If False, don’t match the same ‘on’ value (i.e., strictly less-than / strictly greater-than).

In [14]:
pd.merge_asof(left, right, on="a", allow_exact_matches=False)

,a,left_val,right_val
0,1,a,NaN
1,5,b,3.0
2,10,c,7.0


<IPython.core.display.Javascript object>

In [15]:
quotes = pd.DataFrame(
    {
        "time": [
            pd.Timestamp("2016-05-25 13:30:00.023"),
            pd.Timestamp("2016-05-25 13:30:00.023"),
            pd.Timestamp("2016-05-25 13:30:00.030"),
            pd.Timestamp("2016-05-25 13:30:00.041"),
            pd.Timestamp("2016-05-25 13:30:00.048"),
            pd.Timestamp("2016-05-25 13:30:00.049"),
            pd.Timestamp("2016-05-25 13:30:00.072"),
            pd.Timestamp("2016-05-25 13:30:00.075"),
        ],
        "ticker": ["GOOG", "MSFT", "MSFT", "MSFT", "GOOG", "AAPL", "GOOG", "MSFT"],
        "bid": [720.50, 51.95, 51.97, 51.99, 720.50, 97.99, 720.50, 52.01],
        "ask": [720.93, 51.96, 51.98, 52.00, 720.93, 98.01, 720.88, 52.03],
    }
)
quotes

,time,ticker,bid,ask
0,2016-05-25 13:30:00.023,GOOG,720.50,720.93
1,2016-05-25 13:30:00.023,MSFT,51.95,51.96
2,2016-05-25 13:30:00.030,MSFT,51.97,51.98
3,2016-05-25 13:30:00.041,MSFT,51.99,52.00
4,2016-05-25 13:30:00.048,GOOG,720.50,720.93
5,2016-05-25 13:30:00.049,AAPL,97.99,98.01
6,2016-05-25 13:30:00.072,GOOG,720.50,720.88
7,2016-05-25 13:30:00.075,MSFT,52.01,52.03


<IPython.core.display.Javascript object>

In [16]:
trades = pd.DataFrame(
    {
        "time": [
            pd.Timestamp("2016-05-25 13:30:00.023"),
            pd.Timestamp("2016-05-25 13:30:00.038"),
            pd.Timestamp("2016-05-25 13:30:00.048"),
            pd.Timestamp("2016-05-25 13:30:00.048"),
            pd.Timestamp("2016-05-25 13:30:00.048"),
        ],
        "ticker": ["MSFT", "MSFT", "GOOG", "GOOG", "AAPL"],
        "price": [51.95, 51.95, 720.77, 720.92, 98.0],
        "quantity": [75, 155, 100, 100, 100],
    }
)
trades

,time,ticker,price,quantity
0,2016-05-25 13:30:00.023,MSFT,51.95,75
1,2016-05-25 13:30:00.038,MSFT,51.95,155
2,2016-05-25 13:30:00.048,GOOG,720.77,100
3,2016-05-25 13:30:00.048,GOOG,720.92,100
4,2016-05-25 13:30:00.048,AAPL,98.00,100


<IPython.core.display.Javascript object>

By default we are taking the asof of the quotes.

In [17]:
pd.merge_asof(trades, quotes, on="time", by="ticker")

,time,ticker,price,quantity,bid,ask
0,2016-05-25 13:30:00.023,MSFT,51.95,75,51.95,51.96
1,2016-05-25 13:30:00.038,MSFT,51.95,155,51.97,51.98
2,2016-05-25 13:30:00.048,GOOG,720.77,100,720.50,720.93
3,2016-05-25 13:30:00.048,GOOG,720.92,100,720.50,720.93
4,2016-05-25 13:30:00.048,AAPL,98.00,100,NaN,NaN


<IPython.core.display.Javascript object>

We only asof within 2ms between the quote time and the trade time



In [18]:
pd.merge_asof(trades, quotes, on="time", by="ticker", tolerance=pd.Timedelta("2ms"))

,time,ticker,price,quantity,bid,ask
0,2016-05-25 13:30:00.023,MSFT,51.95,75,51.95,51.96
1,2016-05-25 13:30:00.038,MSFT,51.95,155,NaN,NaN
2,2016-05-25 13:30:00.048,GOOG,720.77,100,720.50,720.93
3,2016-05-25 13:30:00.048,GOOG,720.92,100,720.50,720.93
4,2016-05-25 13:30:00.048,AAPL,98.00,100,NaN,NaN


<IPython.core.display.Javascript object>

We only asof within 10ms between the quote time and the trade time and we exclude exact matches on time. However prior data will propagate forward



In [19]:
pd.merge_asof(
    trades,
    quotes,
    on="time",
    by="ticker",
    tolerance=pd.Timedelta("10ms"),
    allow_exact_matches=False,
)

,time,ticker,price,quantity,bid,ask
0,2016-05-25 13:30:00.023,MSFT,51.95,75,NaN,NaN
1,2016-05-25 13:30:00.038,MSFT,51.95,155,51.97,51.98
2,2016-05-25 13:30:00.048,GOOG,720.77,100,NaN,NaN
3,2016-05-25 13:30:00.048,GOOG,720.92,100,NaN,NaN
4,2016-05-25 13:30:00.048,AAPL,98.00,100,NaN,NaN


<IPython.core.display.Javascript object>

<img src="images/asof_vs_ordered.png" width="900">